In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical

In [4]:
# 빈도수가 가장 높은 1000개의 단어만 추출
(xtrain, ytrain), (xtest, ytest) = reuters.load_data(num_words=1000, test_split=0.2)

C:\Users\poeun\anaconda3\envs\tf2.4\lib\site-packages\tensorflow\python\keras\datasets\reuters.py:148: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
C:\Users\poeun\anaconda3\envs\tf2.4\lib\site-packages\tensorflow\python\keras\datasets\reuters.py:149: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_test, y_test = np.array(xs[idx:]), np.array(labels[idx:])


In [7]:
xtrain.shape # (8982,)
print(xtrain[0]) # 첫번째 뉴스기사

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 2, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 2, 2, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 2, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]


In [8]:
category=np.max(ytrain)+1
category

46

In [10]:
# 단어 수를 100개로 통일 (모자란 부분은 0으로 채워짐. 남으면 제거)
xtrain=sequence.pad_sequences(xtrain, maxlen=100)
xtest=sequence.pad_sequences(xtest, maxlen=100)

In [13]:
len(xtrain[33])

100

In [15]:
ytrain=to_categorical(ytrain)
ytest=to_categorical(ytest)

In [18]:
# 모델 설정
# 임베딩 계층 : 입력된 데이터를 전달 받아서 임베딩 벡터로 변환
# Embedding(단어종류개수, 임베딩벡터차원, 문서최대길이)

model = Sequential()
model.add(Embedding(input_dim=1000, output_dim=10, input_length=100))
# 문서에 등장할 수 있는 단어의 종류가 1000가지 있음
# 1000차원 단어 벡터 -> 임베딩 -> 10차원 단어 벡터 공간

# input_length=100 : 모든 문서에 등장하는 단어 시퀀스 길이가 100이다.


model.add(LSTM(100, activation = "tanh"))
model.add(Dense(46, activation='softmax'))

In [20]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam', metrics=['accuracy'])


In [21]:
model.fit(xtrain, ytrain,
          batch_size=100, epochs=20,
          validation_data=(xtest,ytest))

Epoch 1/20
90/90 [==============================] - 25s 96ms/step - loss: 2.9692 - accuracy: 0.3298 - val_loss: 2.4227 - val_accuracy: 0.3620
Epoch 2/20
90/90 [==============================] - 6s 71ms/step - loss: 2.3968 - accuracy: 0.3576 - val_loss: 2.4133 - val_accuracy: 0.3620
Epoch 3/20
90/90 [==============================] - 6s 62ms/step - loss: 2.3805 - accuracy: 0.3581 - val_loss: 2.1585 - val_accuracy: 0.4715
Epoch 4/20
90/90 [==============================] - 6s 63ms/step - loss: 2.0946 - accuracy: 0.4725 - val_loss: 1.9559 - val_accuracy: 0.4920
Epoch 5/20
90/90 [==============================] - 5s 60ms/step - loss: 1.9063 - accuracy: 0.5088 - val_loss: 1.8258 - val_accuracy: 0.5365
Epoch 6/20
90/90 [==============================] - 6s 62ms/step - loss: 1.8139 - accuracy: 0.5229 - val_loss: 1.7708 - val_accuracy: 0.5392
Epoch 7/20
90/90 [==============================] - 5s 58ms/step - loss: 1.7170 - accuracy: 0.5529 - val_loss: 1.8668 - val_accuracy: 0.5147
Epoch 8/20
9

In [22]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 10)           10000     
_________________________________________________________________
lstm (LSTM)                  (None, 100)               44400     
_________________________________________________________________
dense (Dense)                (None, 46)                4646      
Total params: 59,046
Trainable params: 59,046
Non-trainable params: 0
_________________________________________________________________
